<a href="https://colab.research.google.com/github/PsorTheDoctor/visuomotor-robot-policies/blob/main/data/robomimic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Robomimic quickstart
##Setup

In [ ]:
!git clone https://github.com/ARISE-Initiative/robomimic
!pip install -e robomimic/
import sys
sys.path.append('./robomimic/')

In [ ]:
!sudo apt install curl git libgl1-mesa-dev libgl1-mesa-glx libglew-dev \
         libosmesa6-dev software-properties-common net-tools unzip vim \
         virtualenv wget xpra xserver-xorg-dev libglfw3-dev patchelf

!pip install -q mujoco robosuite

##Download demostration dataset

In [3]:
import os
import json
import h5py
import numpy as np

import robomimic
import robomimic.utils.file_utils as FileUtils
from robomimic import DATASET_REGISTRY

download_folder = '/content/robomimic_data/'
os.makedirs(download_folder, exist_ok=True)

task = 'lift'
dataset_type = 'ph'
hdf5_type = 'low_dim'
FileUtils.download_url(
    url=DATASET_REGISTRY[task][dataset_type][hdf5_type]['url'],
    download_dir=download_folder
)
dataset_path = os.path.join(download_folder, 'low_dim_v141.hdf5')
assert os.path.exists(dataset_path)

ROBOMIMIC WARNING(
    No private macro file found!
    It is recommended to use a private macro file
    To setup, run: python /content/./robomimic/robomimic/scripts/setup_macros.py
)


low_dim_v141.hdf5: 21.7MB [00:02, 7.53MB/s]                            


##Behavior cloning model

In [6]:
import torch
from torch.utils.data import DataLoader

import robomimic
import robomimic.utils.obs_utils as ObsUtils
import robomimic.utils.torch_utils as TorchUtils
import robomimic.utils.test_utils as TestUtils
import robomimic.utils.file_utils as FileUtils
import robomimic.utils.train_utils as TrainUtils
from robomimic.utils.dataset import SequenceDataset

from robomimic.config import config_factory
from robomimic.algo import algo_factory

In [8]:
def get_model(dataset_path, device):
  config = config_factory(algo_name='bc')
  ObsUtils.initialize_obs_utils_with_config(config)

  shape_meta = FileUtils.get_shape_metadata_from_dataset(
      dataset_path=dataset_path,
      all_obs_keys=sorted((
          'robot0_eef_pos',  # end effector position
          'robot0_eef_quat',  # end effector rotation
          'robot0_gripper_qpos',  # paraller gripper joint position
          'object'  # object info
      ))
  )
  # Behavioral cloning model
  model = algo_factory(
      algo_name=config.algo_name,
      config=config,
      obs_key_shapes=shape_meta['all_shapes'],
      ac_dim=shape_meta['ac_dim'],
      device=device
  )
  return model

In [ ]:
device = TorchUtils.get_torch_device(try_to_use_cuda=True)
model = get_model(dataset_path, device=device)
print(model)

##Training loop

In [15]:
def get_data_loader(dataset_path):
  dataset = SequenceDataset(
      hdf5_path=dataset_path,
      obs_keys=(
          'robot0_eef_pos',
          'robot0_eef_quat',
          'robot0_gripper_qpos',
          'object'
      ),
      dataset_keys=(
          'actions', 'rewards', 'dones'
      ),
      load_next_obs=True,
      frame_stack=1,
      seq_length=10,
      pad_frame_stack=True,
      pad_seq_length=True,
      get_pad_mask=False,
      goal_mode=None,
      hdf5_cache_mode='all',
      hdf5_use_swmr=True,
      hdf5_normalize_obs=False,
      filter_by_attribute=None
  )
  print('\n============= Created Dataset =============')
  print(dataset)
  print('')

  data_loader = DataLoader(
      dataset=dataset,
      sampler=None,
      batch_size=100,
      shuffle=True,
      num_workers=0,
      drop_last=True  # don't provide last batch if it's less than 100
  )
  return data_loader


def run_train_loop(model, data_loader, epochs=50, gradient_steps_per_epoch=100):
  model.set_train()

  for epoch in range(1, epochs + 1):
    data_loader_iter = iter(data_loader)
    losses = []

    for _ in range(gradient_steps_per_epoch):
      try:
        batch = next(data_loader_iter)
      except StopIteration:
        data_loader_iter = iter(data_loader)
        batch = next(data_loader_iter)

      input_batch = model.process_batch_for_training(batch)
      info = model.train_on_batch(batch=input_batch, epoch=epoch, validate=False)
      step_log = model.log_info(info)
      losses.append(step_log['Loss'])

    model.on_epoch_end(epoch)
    print('Epoch {}: Loss {}'.format(epoch, np.mean(losses)))

##Run policy training

In [ ]:
data_loader = get_data_loader(dataset_path=dataset_path)
run_train_loop(model=model, data_loader=data_loader, epochs=50)

##Evaluate and visualize

In [ ]:
# Create simulation
import robomimic.utils.env_utils as EnvUtils

env_meta = FileUtils.get_env_metadata_from_dataset(dataset_path)
env = EnvUtils.create_env_from_metadata(
    env_meta=env_meta,
    env_name=env_meta["env_name"],
    render=False,
    render_offscreen=True,
    use_image_obs=False,
)

In [19]:
from robomimic.algo import RolloutPolicy
from robomimic.utils.train_utils import run_rollout
import imageio

policy = RolloutPolicy(model)
video_path = 'rollout.mp4'
video_writer = imageio.get_writer(video_path, fps=20)

rollout_log = run_rollout(
    policy=policy,
    env=env,
    horizon=200,
    video_writer=video_writer,
    render=False
)
video_writer.close()
print(rollout_log)

{'Return': 87.0, 'Horizon': 200, 'Success_Rate': 1.0}


In [20]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open(video_path, 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width=400 controls>
      <source src='{data_url}' type='video/mp4'>
</video>
""")